# Homework: Introduction

## Q2. Indexing the data

In [1]:
import os
import requests
import tiktoken

from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [2]:
os.environ["ELASTIC_PASSWORD"] = "IC1uh2xS7l2Sd+Dk77Zl"

In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
es_client = Elasticsearch(
    "https://localhost:9200",
    basic_auth=("elastic", "IC1uh2xS7l2Sd+Dk77Zl"),
    ca_certs="~/http_ca.crt"
)

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [6]:
index_name = "course_questions"
es_client.indices.delete(index=index_name)
es_client.indices.create(index=index_name, body=index_settings)
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Q3. Searching

In [7]:
query = "How do I execute a command in a running docker container?"

In [8]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [9]:
response = es_client.search(index=index_name, body=search_query)

In [10]:
result_docs = []
for hit in response['hits']['hits']:
    result_docs.append(hit['_score'])

In [11]:
result_docs

[83.29157, 76.03942, 50.81624, 49.25107, 45.703636]

## Q4. Filtering

In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [13]:
response = es_client.search(index=index_name, body=search_query)

In [14]:
result_docs = []
for hit in response['hits']['hits']:
    print(hit['_source']['question'])

How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

In [15]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [16]:
context = ""
for hit in response['hits']['hits']:
    context += ("\n\n" if context else "") + context_template.format(
        question=hit['_source']['question'],
        text=hit['_source']['text']
    )

In [17]:
question = "How do I execute a command in a running docker container?"

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [19]:
prompt = prompt_template.format(question=question, context=context).strip()

In [20]:
len(prompt)

1462

## Q6. Tokens

In [21]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [22]:
tokenized_prompt = encoding.encode(prompt)

In [23]:
len(tokenized_prompt)

322